In [ ]:
import torch
import pandas as pd

: 

In [74]:
from torch import cuda, mps
device = 'cuda' if cuda.is_available() else 'cpu'
device = 'mps' if torch.backends.mps.is_available() else device
print(device)
torch.cuda.empty_cache()

mps


In [ ]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
from model_training import RedditPostDataset
from transformers import AutoTokenizer

# Load the data
df = pd.read_csv('reddit_dataset.csv', usecols=['Title', 'Political Lean'])
df = df.rename(columns={'Title': 'text', 'Political Lean': 'label'})

# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
df['input_id'] = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding='max_length', truncation=True, max_length=128))

# Map the labels to integers (0: Liberal, 1: Conservative)
df['label'] = df['label'].map({'Liberal': 0, 'Conservative': 1})

# Create the attention mask
df['attention_mask'] = df['input_id'].apply(lambda x: [int(i > 0) for i in x])

print(df.head())

# Create the dataset
tokenized_dataset = RedditPostDataset(df)

del df

# Print the first post
print(tokenized_dataset[0])
torch.cuda.empty_cache()

                                                text  label  \
0  No matter who someone is, how they look like, ...      0   
1    Biden speech draws 38.2 million U.S. TV viewers      0   
2                                 State of the union      0   
3              We Should Just Give Poor People Money      0   
4                                  Do it for the Dew      0   

                                            input_id  \
0  [101, 2053, 3043, 2040, 2619, 2003, 1010, 2129...   
1  [101, 7226, 2368, 4613, 9891, 4229, 1012, 1016...   
2  [101, 2110, 1997, 1996, 2586, 102, 0, 0, 0, 0,...   
3  [101, 2057, 2323, 2074, 2507, 3532, 2111, 2769...   
4  [101, 2079, 2009, 2005, 1996, 24903, 102, 0, 0...   

                                      attention_mask  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...  
4  [1, 1,

In [14]:
from torch.utils.data import DataLoader

# split the dataset into a training and evaluation set
train_size = int(0.8 * len(tokenized_dataset))
eval_size = len(tokenized_dataset) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(tokenized_dataset, [train_size, eval_size])

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

# Erase old dataset
del tokenized_dataset

# Check the size of the training and evaluation sets
len(train_dataloader), len(eval_dataloader)
torch.cuda.empty_cache()

(41, 11)

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from torch.optim import AdamW

# Define the optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [17]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler('linear', optim, num_warmup_steps=0, num_training_steps=num_training_steps)

In [75]:
from tqdm.auto import tqdm
from torch import nn

loss_fn = nn.CrossEntropyLoss()

progress_bar = tqdm(range(num_training_steps))

# Empty cache (saving memory)
torch.cuda.empty_cache()

# Train the model
model.train()
model.to(device)
for epoch in range(num_epochs):
    for batch in train_dataloader:
        torch.cuda.empty_cache()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optim.step()
        lr_scheduler.step()
        optim.zero_grad()
        progress_bar.update(1)

  0%|          | 0/123 [09:04<?, ?it/s]


In [71]:
import evaluate

# Evaluate the model
metric = evaluate.load('accuracy')
model.eval()
with torch.no_grad():
  for batch in eval_dataloader:
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.7427293064876958}

In [ ]:
# Save the model
torch.save(model.state_dict(), 'model_state_dict.pth')

In [21]:
# Load model from model_state_dict.pth
model.load_state_dict(torch.load('model_state_dict.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [53]:
transcript = ''.join(['Amen. ', 'Okay, well it is good to be back with you. ', 'I wanted to just give a point of information. ', "I had been starting the midweek review during this Corona lockdown from the Corona Cave because I missed getting to update you all, but I've recently started another podcast with a couple of friends of mine and I have a link up to it now at my website, jdcoke.com and it's with a group of guys and women, it was called the Stan Ferm and Faith podcast and it's a good friend of mine, the Reverend Matt Kennedy and dear friend, the Reverend Nick Lanan. ", "And I've been doing that on Wednesdays and it just frankly has been a lot of output and so I haven't gotten around to the midweek review this past two weeks, but I will continue to put those up as the situations present themselves with the information but I do commend that to you. ", "I will have a link, you can see the slide if you're looking along and it's a little bit of a, an attempt to talk about the current events within particularly the Anglican church in North America and sort of our denomination and our church as we continue to walk through the creation of this new entity, the Anglican church in North America. ", "So that's just a small plug to you all and I hope that you'll check it out. ", "We've been enjoying it and these are two dear friends of mine and faithful pastors and articulate and courageous theologians and people with whom I'm grateful to be associated. ", 'So there we go. ', 'He was at Old Club, read the book that William F. ', 'Buckley talked about his run for mayor in the 60s in New York and he said the famous quip was what would they do? ', 'What would he do if he won? ', "And it'd be like demand to recount. ", "It's like, was it, was it Groucho Marx who said that he wouldn't wanna join any club that would have him as a member? ", "So again, and I don't know why I'm saying that other than, I'm grateful to be counted among Mottley Crue, as it were, of courageous standing firm theologians. ", 'But there we go. ', 'Along those lines, I wanted to draw you. ', 'This is a tweet from a guy named Jeff Walton who runs something called the Institute for Religion and Democracy, which is kind of a clearing house or sort of an aggregator for all sorts of different news on the church and the American church in particular. ', "And I believe he's an Anglican. ", "I'm not sure, I don't know him personally, but he does a reporting, I just ran across this tweet, which is recent news and he says this, Texas Supreme Court issues ruling in Fort Worth Episcopal Property dispute. ", 'Colin finds it withdrawing Anglican faction as the diocese versus appeals court. ', 'This is a major ruling that resolves a 12 year dispute. ', "Well, I haven't had yet had time and I may not get to read the entire court, the judgment, but the long and short of it is that the ACNA, the diocese of the formerly known, the diocese formerly known as Prince, formerly known as the Episcopal Diocese of Fort Worth has withdrew and joined the Anglican Diocese, the Anglican church and there was a legal dispute that's lasted 12 years now in which some good friends of mine and some dear fine churches have been in the midst of this. ", 'And the Supreme Court of Texas has ruled. ', "I spoke with someone in one of the parishes who said that there's a possibility, of course, for appeal to the US Supreme Court, but the US Supreme Court has shown itself to be very reluctant to intervene in these matters and ends in fact a couple of years ago passed over some similar cases that they could have weighed in on and is there want, they decided not to take up the case. ", 'So this may be the end of that fight and of course for some of you know us in South Carolina are in a very similar position. ', "And of course we're hopeful, I think, because both sides would be hopeful for a victory but certainly we're sitting here having walked through this litigation for many years and are hopeful that this is a sign of things to come. ", 'It may not be and of course people have been prepared and many people have already had to leave their buildings and turn over their property and have, the Lord has been faithful and merciful to them in light of that. ', "Although there's something to be said for the history, the blood, sweat and tears and equity that has been put into a church that you don't want to lose. ", 'So anyway, that was just of interest. ', 'I know that there are disappointed people on the other side. ', 'There are an elated people on the AC&A side and at the very least I know that at the end of the day, the end of this sort of fight has to be a some sense of relief. ', 'I would imagine 12 years is a long time to live an uncertainty on whatever side you may. ', "Okay, I also wanted to say I've been teaching this prayer book class, some of you know, on the AC&A Book of Common Prayer and I've really enjoyed it and it's actually kind of inspired me to dig even deeper into my own personal life with respect to the prayer book and it's used in my family and of course I use it in my devotional life but I wanted to draw you to a resource. ", 'This is the dailyoffice2019.com and it refers to 2019 is not the year but to the prayer book and I commend this resource too. ', "You go there and you can put a little, you know, a bookmark on your web browser or whatever and it's just a wonderful resource. ", 'They have all of the readings from every day. ', "They have morning evening, I mean morning, midday, evening, compliment, they have family prayer at midday which I'm showing you here and it's really a wonderful thing and evidently they're putting together an app for both Android and Apple phones so that's exciting but I just wanted to commend this to you because I think of course the book, you know, there's no substitute for, ain't nothing but the real thing baby, right, for no substitute for actual books but if you need a resource to help you begin, this is as good as any as I've found. ", "If you know any other ones or better ones, I'm always looking for one so please let me know but for now I believe this should suffice for most of our needs with respect to the daily office as it's known. ", 'Okay, well I wanted to talk today about some just general ideas that I think we need to continue to mull over as Christian people and they have to do with, well of course the presenting issue of the coronavirus but not just that but it\'s the idea of what is human life, what are our responsibilities to our neighbors with respect to the pandemic and the fear of transmission and all of the various discussions going around particularly church and how we should be modeling what it is we believe and why to the world during this quote unquote reopening time and to that end I wanted to first look at an article from the New York Times because it was fascinating to me, it\'s by a guy named Thomas Friedman, if you read the New York Times and you\'ll know he\'s frequent contributors, one of their columnists but he has this fascinating article called, "Is Trump Challenging Mother Nature to a Dual? ', 'He can\'t beat her coronavirus,." And so we\'re gonna begin, you listener, you\'ve already picked up on the sort of the pronoun usage here is a famous and oft repeated refrain that the earth is our mother, that we need to recognize sort of our role within the cosmos as, you know, ants on a hill as we are within a closed system within which the universe speaks mercilessly and coldly and mother earth has never lost in 4.5 billion years as Thomas Friedman will say, I\'ll just read a little bit about this from this article., "When Easter as a coronavirus was rapidly spreading, NPR\'s weakened all things considered, carries excerpts, that was my NPR voice"" "cared excerpts from sermons from across the country. ', 'I was particularly touched by the way presiding Bishop Michael Curry ended his talk at Washington National Cathedral singing, he\'s got the whole world in his hands, he\'s got the whole world in his hands,." We\'ll just leave that with it, where it is., "It was a powerful close that left me thinking, just substitute she for he and you\'ve defined the core problem we\'re facing. ', 'For the first time in the life of our generation of human species, mother nature has the whole world in her hands. ', 'The entire planet is collectively facing the same challenges from the same coronavirus at the same time,." Now, I want to just stop right there, because you can see where this is going, that all of the hubris of the human person over against mother earth, the cosmos nature, will be seen as futile, and that we should throw our hands up at any idea that somehow we may triumph over mother nature. ', 'And so there is obviously something kind of both because the intersection of technology and communication and just our own sort of awareness that we are aware of the fact that from New Zealand to Canada, to Siberia, to Brazil, we are all dealing with the same thing at the same time. ', 'And that is quite a novel awareness at the very least. ', 'But I want to continue on, because he speaks to something more fundamental that I think Christian people can reflect upon. ', 'He says this"" "Mother nature is not only all powerful, she\'s also unfeeling. ', "Unlike that merciful God that most humans worship, mother nature doesn't keep score. ", 'She can inflict her virus on your grandmother on Monday and blow down your house for the tornado on Wednesday and come back on Friday and flood your basement. ', 'She can hit you in the spring, give you a warm hug in summer and hammer you in the fall,." Well, the long story short of his column is that we need to just be aware that we will lose to this and prepare ourselves to be humbled by the unflinching march of Mother Nature. ', "But I wanted to just point out this idea that this unlike that merciful God that most human worship, human worship, Mother Nature doesn't keep score. ", 'And exactly what does he mean by that? ', "Because a merciful God who keeps score would show mercy across the board, that doesn't make sense to me when he's trying to argue that if Mother Nature doesn't keep score and so is therefore then she's seemingly capricious, then that's to argue that the judgments that are non capricious sort of score keeping, quote unquote, merciful God, the judgments that are levied do somehow make sense, which of course goes in line with some sense, some religious ideas, karma most notably or even some sort of deterministic sex of Islam or just some conceptions even of God from in some Christian, heretical views, whereby God can be seen tied one to one to specific judgments and to specific calamities outside of that which he's revealed in scripture. ", "But the Christian message, and we've talked about this from the beginning of the coronavirus, is this simultaneous thing that the world is broken and flawed and death ridden on account of sin and that God in his mercy has not removed us from this world of sin, death in the devil but has given us the means by which we can stand in the midst of it and understand to some degree where he is to be found even in the midst of the coronavirus and so like Jesus on the Mount of Olives, like Jesus on the cross for that matter, there's never a question as to whether God was good or where he was but simply there was a sense of well for Jesus, a forsakenness towards the end, a fear of this before he finally yells it's finished on account of him as Paul says in 2 Corinthians, he knew no sin becoming sin so that in him we may become the righteousness of God. ", "And so I just thought this was fascinating because the way that people speak about God is very much the way that he's speaking about mother nature in this respect in that it's unfeeling, unthinking, unknowable, capricious and ultimately deadly and that is not a Christian way to speak about God. ", 'And beyond that, the reality of sin, death in the devil is that nature as it were, as Thomas Hobbes said, red and tooth and cloth, that the creation that has been subjected to futility, futility as Paul says is groaning, amoning for redemption and Jesus has the first fruit that new redemption points towards the ultimate consummation, the eschaton of what God will do when he redeems the new heaven and new earth but until then, as it were, our nature will continue to crumble, our bodies will continue to give way, our minds will continue to fade as Paul writes that we outwardly, although we are wasting away inwardly, we are being renewed by grace upon grace and that this is the Christian witness that yes, nature, the viruses, the cancers, the sun spots, the old age, the broken bones, the pestilence, the disease, the famine, these things are part of the broken world that continue to win, they continue to ultimately defeat but Christians stand firm and courageous in the midst of that and to a certain degree, fight back against it. ', "I mean, there's a lot of courage that Christian people and people for that matter but Christians in particular have taken knowing that there is no ultimate defeat of this outside of faith in Christ in the hope of the resurrection and so therefore you could be a little more cavalier or a little more ambitious or adventurous some with your life with respect to the dangers set before you, knowing that God Christ has gone ahead of you and that what ultimately do you have to fear when the days of your life are numbered in his hands and in his heart and mind? ", "And so again, I just wanted to point this out because this is a strange essay for me really because this dichotomy between the supposed belief in God as this merciful scorekeeper and then the fear of the unknown capricious nature pointed to as always is the case with the New York Times as least as far as I can tell outside of Ross Douthat's writing a misunderstanding of what actually is taught and preached in gospel-centered, biblically-based churches but there we go. ", "We'll just continue, I'm sure if Nicholas Christoph, I mean, Thomas Friedman wants to call me, I'd be happy to help him work through his confusion so I'll wait on that call for a while. ", "Okay, well let's move now. ", "You may have seen, if you haven't, then you, well, I know you have, dear listener, about the sort of contrary controversy going around the discussion about reopening churches in particular and so there was this much, much debated CDC guidelines about churches in particular that got a lot of pushback from a lot of different churches in part because it seemed to be singling out churches for specific sort of regulatory guidelines that weren't general, sort of the principle of general equity, the generally equitable across all places. ", "If you can't sing, period, then you can't specifically, then there's making any difference to say you shouldn't specifically sing in church if you can't actually go sing in your local group, karaoke night or whatever it is you do. ", 'You know your local Rocky Horror picture shows sing along that you do once a week. ', "So if there's some general guidelines that are laid out and there's kind of an agreed upon expectation of these are best practices, well, then churches are more than happy to join into them but there was some pushback on the idea that churches would be specifically targeted and I've really been wrestling with why this is the case. ", "We'll go quickly down here for a minute. ", "You'll see that in Minnesota, Minnesota, thousands of churches, it's an interesting bedfellows between the LCMS, the Lutheran Church, Missouri Senate and the Roman Catholic Church, which could sort of understand the Louis V, one opposite sides of the great Reformation divide. ", "Nevertheless, come together and I believe even the Council of Rabbis, if I'm not mistaken, but at the very least, a diverse group of people who want to get together on Sunday, Saturday or Sunday on the weekend for religious observance have come together and are, I guess we're gonna find out today, if you're listening this on the 24th, whether or not they both met in defiance of the lockdown, whether or not Trump, as you heard probably yesterday, said that churches deem them essential but of course this brings into, as we've talked about before, the sort of Federalists break down of whether or not the Executive can override the governors and where does the power lie to talk about any of this stuff, which of course we have governors all the way from Ron DeSantis to Gretchen Whitmer, who are blazing new trails in our experiment in separation of powers and our experiments in self-governance and our experiment in representative democracy and federalism and all of these things are relatively interesting to me. ", "I mean, I'm in church and I've been grateful, you know, our Bishop and the various rectors in our diocese have voluntarily assumed, you know, sort of the height of caution with respect to reopening. ", 'We are going to reopen. ', "We have not ever been told we couldn't, but we are attempting to walk the line of navigating love for our neighbor and care for and concern for those likely or possibly affected by exposure to this. ", 'Balancing that with the Christian necessity to continue to meet, to continue to not stop fellowshipping as the Bible enjoins us to do. ', "And that really gets down to the heart of it for me and I think you should probably consider these past couple of weeks and I think it's a good exercise. ", "Like what have you actually missed when you haven't been in church? ", "I mean, Zoom, there's some limitations to Zoom. ", "There's also some benefits. ", "Some of the, you know, people have been able to maximize the sound, you know, people have been able to come back to various parts of the sermon in ways that if you didn't have it recorded or you didn't have it, you know, you didn't have time to come or you felt, you know, you were distracted on Sunday. ", "I mean, there's all sorts of kind of benefits to our digitized life, but there's some real drawbacks too. ", "And I think it's a good thing to reflect upon because what I have found at least in my life, two things. ", "I think that from the outside, there's a real confusion, which makes sense if you're not a Christian or if you're not a Christian who, you know, you know, these research, Pew research and Gallup polls that we always reference where they talk about, you know, X number, percentage of Christians. ", '50% of people are Christians or 72% of people are Christians or some sort of thing like this. ', "And then they dig into it and it's like, well, of those 50% who said they were Christian, 10% actually go to church more than once a year, you know, or something like this. ", "And so let's just say that we had some sort of poll that had some kind of, some sort of engagement and it said, do you actually go with few exceptions? ", 'Is this something that you would consider to be a priority in your life? ', 'When you list the priorities in your life or your involvement with church, you know, first, I mean, frankly, is your, you know, love of God, love of family, love of self, you know, this sort of, is that the hierarchy or is it some other hierarchy? ', 'And I would imagine that among the people they would argue that God and their church was primary, that there has been a deep sense of loss and a deep sense of longing, I can speak for myself. ', "And I've been trying to reflect upon what that is and it's all sorts of things. ", "And we're gonna see as we open back up what sort of hits us as more or less meaningful once we reengage. ", 'But I do know that there is something that has been profoundly sort of impressed upon me to the necessity of corporate worship, the strength that is given when you actually look next to you and see a harried family who has nevertheless come together at the same time to hear this, sing the same songs and say the same prayers and stand firm in the same faith. ', "That means the world and, you know, whether it's 10 people or it's a thousand people, it really doesn't matter because when you know that you're not only not alone, but that the Lord has called out of a motley group of people. ", 'So all shapes and sizes to come together at one point in time and space, never to be repeated, to sing his praise, to hear his word, to confess their sins, to receive absolution, to be strengthened once again by the means of grace and word and sacrament, this cannot be virtually undertaken. ', 'And I think the fact that we have been able to do it for these numbers of weeks has simply been because of the promise of a return at some point, masked or not. ', 'You know, HDC hazmat suits or not, that we will wear whatever, we will say whatever, we will sanitize whatever needs to be done in order to return back to the way of embodied worship, which has marked the worship of God, Father, Son, and Holy Spirit, since He called Abraham out of Ur. ', 'This is what we have been doing. ', 'And so I am excited about coming back. ', "We are going to be taking all the proper precautions as we have up until now, and who knows how long we'll have to do them. ", 'And we pray for anyone and everyone who has been affected specifically by this virus, not indirectly through shut down, of course we pray for them too. ', "But I do know that it's been not surprising to me that among those who are the most excited about coming back to church are the quote, quote, people who are the most vulnerable, you know, some of the people that I know that are rather ill or who are aged or who are, you know, who otherwise should perhaps consider themselves in the high risk category are going to be lining up to worship God and Spirit and Truth once again. ", "And I'm grateful for that. ", 'And I of course will take every and all precautions to make sure that they are protected in the midst of their worship. ', "Certainly more so, I would imagine than some of the local hardware stores do, although I'm sure that they are doing the best they can as well. ", "Okay, well, we're gonna turn now to a really interesting sort of development this week. ", "I don't know if you saw it, but they, in the end of her life, the woman Norma McCorvey, who was known as Jane Rowe, she was filmed for a documentary. ", 'She died in 2017 at the age of 69, and it came out in this documentary, well, it just read the headline from the UK Guardian"" "Rough vs. ', 'Wade Planiff admits abortion rights reversal, quote"" "was all an act in new film,." Norma McCorvey, most notable for being the Planiff known as Jane Rowe in the 1973 landmark Supreme Court case, Roe vs. ', 'Wade, that led to abortion becoming legal in the United States, made a stunning omission just before her death in 2017. ', 'It has emerged. ', 'This is my deathbed confession, she explained. ', 'And she goes on to say that she essentially was coerced and then ultimately compensated for her story where she ultimately changed her mind and became an anti-abortion sort of, she became a proponent for anti-abortion. ', "And the interesting part, I don't know if you know this, but she actually, in the midst of her wanting to get an abortion, was actually had her child and gave the child up for adoption. ", 'But nevertheless, it was a big, obviously you can imagine, it was a big thing. ', 'I was a young child at the time when this was going on, but I can imagine those of you who lived through it. ', 'It was news, the very woman who was at the heart of this great divide changes her mind. ', "And that's good TV, that's good rhetoric. ", "And I'm not surprised that people picked up on that. ", "Now, there's a lot about this that I think is kind of interesting from a very ancillary perspective about her life and people who knew her and evidently, God bless her, but she had sort of a troubled life and there was sort of some wavering back and forth throughout her life. ", 'And she was a complicated person. ', 'And we understand complicated people. ', 'Certainly you, dear listener, and we give lots of grace to them. ', 'Nevertheless, I thought it was fascinating, the response to this, because somehow, even if she did change her mind, even if she were paid by some unscrupulous-- because that would be an unscrupulous thing to do, to pay someone to essentially lie, if that is in fact what happened. ', 'That somehow this exposed some deeper sense of deviancy or some duplicity at the heart of the anti-abortion movement. ', "That's what I found fascinating, because whether she lied or not, whether there was some unscrupulous sort of anti-abortion crusaders that were using dishonest means for a supposedly good gain. ", "All of these things, which are questionable, if not just outright abhorrent from a sort of Christian perspective, doesn't change the fact that what is actually being argued fundamentally is a non-- there's no middle ground about this debate. ", 'And it is probably the most illustrative of a worldview divide, as our friend Al Moller likes to say, that if you-- because all of the arguments, all of the arguments about the life of the mother and sort of being pro-life at every stage and wanting to take care of people in their distress, all of these are good and true arguments. ', "There's no one that I know that is anti-abortion, whatever, remotely say that the life of a poor, unwed mother, or a scared young girl, or any of the actual mothers, in this case, are to be taken for granted. ", 'No one I would say that. ', 'They would simply additionally say what the baby also needs to be taken into account. ', 'That this is not a one-person conversation. ', 'There are two people involved in this conversation. ', "And whatever we can do to protect the lives of both people, whether that's through education or sort of financial support, or putting babies up for adoption, or emotional, psychological care, all of the things which all of the good adoption agencies that I personally know, because lies and lies, you know, have been involved in adoption, or are continuing to be involved in adoption agencies. ", 'Every one of them to an agency understands the lifetime of care that needs to go into protecting and helping a woman heal from a quote-unquote unwanted pregnancy, particularly when she has the baby. ', "And so the idea that there is a sort of-- a-- what's the word I'm looking for here? ", 'That there is a dishonesty at the heart of an anti-abortion person when it comes to the care of the mother. ', "It's just false. ", "And it just goes to show how few-- we're sort of all in echo chambers, because how few people have actually-- who are sort of anti-abortion, sort of pro-plan parenthood people have actually ever sat down with someone who has done everything they can to care for the life of their adopted child's mother, or something along these lines. ", 'But I want to just point that out that David Hirani-- Harsani, excuse me, of the National Review just wrote about this. ', 'And I just wanted to read it, because I thought he nailed it. ', 'He said, according to a new documentary, Norma McCorvey, aka Jane Rowe of Roe versus Wade, made a deathbed confession that her pro-life conversion and activism was all an act. ', 'Funded by anti-abortion organizations, I had a few off the record conversations from the Corvey over the years, and nothing in those chats felt scripted to me. ', 'You can never really know, I suppose. ', 'My guess is that McCorvey was a troubled woman thrown into the middle of one of the most contentious Supreme Court cases in history, who shifted her positions in search of a public peer approval. ', 'Whatever the case, her later stances have no bearing on the debate over abortion. ', 'The fact that pro-life groups paid McCorvey to speak is not a big revelation nor a big deal, nor do her vacillating claims tell us anything valuable about the constitutional validity of Roe versus Wade or the morality of dispensing with human life for convenience. ', "Yet Laura Bassett at GQ would have her readers believe that McCorvey's deathbed admission tells us everything Americans need to know about the pro-life movement. ", "Bassett's baffling and ahistorical central claim is that pre-Ragan Republicans were pro-abortion because they were racist, and post-Ragan Republicans were pro-life also because they were racist, which is quite convenient. ", "Well, getting to the whole history of this-- we've spoken about this a lot-- but I think the argument in the GQ article, which you can read if you want you and the 14 other people that subscribe to that magazine anymore, but you can read it. ", "And the problem is that the fundamental question about which we've been speaking is always just subsumed under power dynamic or racial talk or something other than the actual argument, which is whether or not it's a human life. ", "And that's the argument that will continue to simply be more and more clear as ultrasound technology gets better, as younger and younger babies can be saved in vitro, as operations on 20 week and 18 week and 16 week and 10 week as genetic-- as much as fearful as genetic manipulation, CRISPR technology and things for embryos is I'm sure that there'll be some positive benefits for it, too. ", 'So babies with six celled babies that have spina bifida will be able to be healed in utero, things like this. ', 'That will just continue to be larger and larger and larger. ', "And the actual discussion will become clear and clear and clearer, which is that either we are creatures in the hands of a god given certain roles and responsibilities whereby we are constrained by our bodies, by what they do or can't do, by what they produce or cannot produce, or we are, as the serpent promised, even the garden, our own gods, like God, and we will live forever. ", 'And that will simply be the two alternatives, which will continue to be put as stark relief in front of us. ', 'And quote unquote"" "mother nature will continue to mock our pretensions to be God, and our consciences will continue to scar us as our impending deaths continue to draw out of us a sense of dread, doom, and disillusionment, which, of course, has been met and fulfilled and redeemed in the death and resurrection of Christ,." OK, well, I\'ve got some more things to say. ', "I'm going to actually have time for midweek review this week. ", 'But I wanted to leave you with a happy idea. ', "We're going to talk a little bit more about abortion on Wednesday, because we're going to talk about how few doctors will admit even if they are abortionists, to being abortionists, and how that points to a deep sense of shame, a godly shame for doctors who take the Hippocratic oath of do no harm in actually killing babies, and some of the things that they express are just heartbreaking, but also blood curdling. ", "And we're also going to talk a little bit about the increase in mental health problems during the COVID lockdown, and how the church in particular and Christian people can be praying for that, and how we can be ministering as agents of the gospel to those who are hurting around us. ", 'But I wanted to leave you with one final thing. ', 'We spoke last week about Ross Dout and his book about decadence. ', 'Well, I found another review, and I thought it was really fascinating, because it has some overlay to coming full circle to the AC&A and our Anglican church going forward in the world. ', 'He says"" "Ross Dout that has a vision of America and its grim,." And so let\'s see how the review goes. ', 'By this point, even the most patient reader will have questions and doubt that anticipates many of them. ', 'How can he say that ours is a static culture in the age of connectivity, which has accelerated change and turned ours into ever more, quote"" "liquid societies, an illusion. ', 'It has simply accelerated the circulation of the same tired cultural tropes, leaving us with wheels that turn ever faster in a ditch. ', 'What about the political radicalism and rage exacerbated by social media? ', 'Probably just, quote"" "a kind of digital age play acting, in which young people dissatisfied with decadence pretend to be fascist and Marxist on the internet, reenacting the 1930s and 1960s with fewer street fights and more memes. ', 'One chapter concludes, quote"" "It\'s possible that Western society is really leaning back in an easy chair, hooked up to a drip of something soothing, plain and replaying an ideological greatest hits tape from its wild and crazy youth, all riled up in its own imagination, and yet in reality, comfortably numb,." Like Pink Floyd, of course. ', "But so there's a lot to be said there. ", 'I think-- I was hard not to read this and just watch it. ', "They're even remaking Scooby-Doo. ", 'Have you seen this? ', "It's called Scoob. ", "They're remaking Top Gun. ", "They're remaking-- they're going to remake Hoosiers. ", "I mean, they've already made 75 Rockies, which are a pretty terrible Rambo. ", "There's seven Rambo's now. ", "They're just going to keep remaking things, because people do seem to be sort of bored. ", 'The art world is just decadent, flabby, soft. ', 'And I think this does leave us. ', 'And we see in the West, open to being overtaken. ', "Again, you don't have to just be a genius or even that insightful to see some similarities with the decline of the Roman Empire, and it's sort of decadence and sort of boredom. ", 'What do you do when everyone-- you killed everyone, everyone worships you, or everyone is bread and circuses? ', "That's all you got. ", 'And so then there was this sort of this takeover. ', "So I don't think that we have lost-- Christianity has lost the way, but I do think that a lot of Western Christianity is playing into-- you see this. ", 'You see how few sort of quote unquote, "progressive"" churches are going to open up ever again, out of love for neighbor. ', 'We will never have church again. ', 'I mean, this is the sort of excitement that many churches have meeting. ', 'But I just wanted to leave you. ', 'This is not bad news. ', "I'm not going to read this whole thing. ", 'But he says it to hope for the West is immigration from the east and most notably from Africa. ', 'And he talks specifically about the way that African Christians are going to rejuvenate the empty churches of the now post-Christian West. ', 'And I have seen this firsthand. ', 'When we were in Vienna, Austria, half of our congregation was only 100 or so seat church. ', "It wasn't very large. ", 'But more than half, I would say, were non-native Austrian, certainly not Westerners. ', 'And so we have people from Kenya and Ghana and Nigeria and South Africa and all sorts of places who had brought with them an excitement for life, a joy in the gospel, a hope for a better life for the future, that was shocking and inspiring and convicting for some sort of disillusioned Westerners. ', 'And so this is where Ross Dout that puts, he says that until we can find ways to sort of further explore the world, our real hope lies in being rejuvenated by people who actually seem to have tasted something of the hope that Christ has set before them. ', 'And I would hope to consider myself in that. ', 'I look at my children. ', 'I look at the people around me in church. ', "And I know that despite some of the confusion and sort of the difficulties that this world seems to be presenting itself in the future, that I trust that they are not only going to be in the Lord's hand, but that he will guide and lead them. ", 'And even if the world looks different, they will live with a certain sense of hopeful joy, as we say in the baptism, right, that there will be a gift of joy and wonder in all your works. ', "And at the very least, there's a gift of joy and wonder and the fact that God himself has condescended to come down into our sinful world and to redeem us by his own blood and therefore given us a hope upon which we stand, which carries us in the midst of all of the uncertainties of life and gives us a joy and a peace that the world does not know. ", 'And that is the hope that is being infused by many Africans as they immigrate to the West. '])

# Now that the transcript is one long string, split into chunks of 128 tokens each (use nltk.word_tokenize() to split the transcript into tokens).
import nltk
tokens = nltk.word_tokenize(transcript)
chunks = [tokens[i:i+128] for i in range(0, len(tokens), 128)]
# recombine the tokens into strings
chunks = [' '.join(chunk) for chunk in chunks]
print(chunks)

torch.cuda.empty_cache()

scores = torch.tensor([[0,0]], dtype=torch.float32)
max_conservative = 0
max_liberal = 0
most_conservative = ""
most_liberal = ""
print(scores)
with torch.no_grad():
  for chunk in chunks:
    # Suppose `tokenizer` is your DistilBERT tokenizer and `model` is your DistilBERT model
    encoding = tokenizer.encode_plus(
        chunk,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    encoding = {k: v.to(device) for k, v in encoding.items()}

    # You don't need to manually create the mask, encode_plus will do that
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Make sure the input tensors have the batch dimension
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    if outputs.logits[0][0] > max_conservative:
      max_conservative = outputs.logits[0][0]
      most_conservative = chunk
    if outputs.logits[0][1] > max_liberal:
      max_liberal = outputs.logits[0][1]
      most_liberal = chunk
    scores += outputs.logits


# To get the model's predictions or logits, you can access `outputs.logits`
scores = scores / len(chunks)
pred = "Conservative" if torch.argmax(scores) else "Liberal"
print("Most conservative chunk: ", most_conservative)
print("Most liberal chunk: ", most_liberal)
print(pred)

["Amen . Okay , well it is good to be back with you . I wanted to just give a point of information . I had been starting the midweek review during this Corona lockdown from the Corona Cave because I missed getting to update you all , but I 've recently started another podcast with a couple of friends of mine and I have a link up to it now at my website , jdcoke.com and it 's with a group of guys and women , it was called the Stan Ferm and Faith podcast and it 's a good friend of mine , the Reverend Matt Kennedy and dear friend , the Reverend Nick Lanan . And I 've been doing that on Wednesdays and it", "just frankly has been a lot of output and so I have n't gotten around to the midweek review this past two weeks , but I will continue to put those up as the situations present themselves with the information but I do commend that to you . I will have a link , you can see the slide if you 're looking along and it 's a little bit of a , an attempt to talk about the current events within p

In [68]:
# Testing things out: load in reddit data, and filter out the posts without body text
import pandas as pd
import numpy as np

reddit_data = pd.read_csv("reddit_dataset.csv", usecols=['Text', 'Political Lean'])
reddit_data = reddit_data.dropna(subset=['Text'])

# Further filter out the posts that contain links instead of text
reddit_data = reddit_data[~reddit_data['Text'].str.contains("http")]

reddit_data = reddit_data.rename(columns={'Text': 'text', 'Political Lean': 'label'})

# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
reddit_data['input_id'] = reddit_data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding='max_length', truncation=True, max_length=128))

# Map the labels to integers (0: Liberal, 1: Conservative)
reddit_data['label'] = reddit_data['label'].map({'Liberal': 0, 'Conservative': 1})

# Create the attention mask
reddit_data['attention_mask'] = reddit_data['input_id'].apply(lambda x: [int(i > 0) for i in x])

reddit_data = reddit_data.reset_index(drop=True)

# Create the dataset
tokenized_text_dataset = RedditPostDataset(reddit_data)

# Look at the first example
print(tokenized_text_dataset[0])

eval_dataloader = DataLoader(tokenized_text_dataset, batch_size=128, shuffle=False)

{'input_ids': tensor([ 101, 2040, 3427, 1996, 2110, 1997, 1996, 2586, 2197, 2305, 1998, 2054,
        2001, 2115, 5448, 1029,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0